In [ ]:
import pandas as pd
import numpy as np
import time 
import matplotlib.pyplot as plt
from scipy.linalg import qr, solve
from memory_profiler import profile
def ALS(W,H, A, A_test, Omega, test, k, lambda_, max_iter, tol=1e-4):
    m, n = A.shape
    A_filled = W@H.T
    RMSE = [np.sqrt(np.mean(((A_test[test] - A_filled[test]) ** 2)))]
    objective=[np.linalg.norm(A[Omega]-A_filled[Omega])+lambda_*(np.linalg.norm(W,"fro"))+np.linalg.norm(H,"fro")]
    start_time = time.time()
    times=[0]
    for iteration in range(max_iter):
        for i in range(m):
            observed_j = np.where(Omega[i, :])[0]  # Observed items for user i
            H_Omega_i = H[observed_j,:]  # Submatrix of H for observed entries
            A_Omega_i = A[i, observed_j]  # Observed ratings for user i
            W[i] = np.linalg.inv(H_Omega_i.T @ H_Omega_i + lambda_ * np.eye(k))@ (H_Omega_i.T @ A_Omega_i)
        for j in range(n):
            observed_i = np.where(Omega[:, j])[0]  # Observed users for item j
            W_Omega_j = W[observed_i,:]  # Submatrix of W for observed entries
            A_Omega_j = A[observed_i, j]  # Observed ratings for item j
            H[j] = np.linalg.inv(W_Omega_j.T @ W_Omega_j + lambda_ * np.eye(k))@ (W_Omega_j.T @ A_Omega_j)
        A_filled = W @ H.T
        RMSE_new = np.sqrt(np.mean((A_test[test] - A_filled[test]) ** 2))
        objective.append(np.linalg.norm(A[Omega]-A_filled[Omega])+lambda_*(np.linalg.norm(W,"fro"))+np.linalg.norm(H,"fro"))
        RMSE.append(RMSE_new)
        times.append(time.time() - start_time)
    elapsed_time = time.time() - start_time
    return A_filled, np.log(objective), elapsed_time, times,RMSE

def CCD_MC(W,H, A, A_test, Omega, test, k, lambda_, max_iter, tol=1e-4):
    m, n = A.shape
    A_filled = W@H.T
    RMSE = [np.sqrt(np.mean(((A_test[test] - A_filled[test]) ** 2)))]
    R = np.zeros_like(A)
    R[Omega] = A[Omega] - (W @ H.T)[Omega]  
    start_time = time.time()
    times=[0]
    objective=[np.linalg.norm(A[Omega]-A_filled[Omega])+lambda_*(np.linalg.norm(W,"fro"))+np.linalg.norm(H,"fro")]

    for iteration in range(max_iter):
        prev_R=R.copy()
        for i in range (m):
            # \i=np.random.randint(m)
            observed_j = np.where(Omega[i, :])[0]  # Observed columns for row i
            for t in range(k):  # Iterate over latent factors
                prev_W = W[i, t]
                H_tj = H[observed_j, t]  
                W[i, t] = (R[i, observed_j] + W[i, t] * H_tj) @ H_tj / (lambda_ + (H_tj.T @ H_tj))
                R[i, observed_j] -= (W[i, t] - prev_W) * H_tj
        for j in range (n):
            # j=np.random.randint(n)
            observed_i = np.where(Omega[:, j])[0]  # Observed rows for column j
            for t in range(k):  # Iterate over latent factors
                prev_H = H[j, t]
                W_it = W[observed_i, t]  # Latent factor t for observed entries
                H[j, t] = (R[observed_i, j] + W_it * H[j, t]) @ W_it / (lambda_ + (W_it.T @ W_it))
                R[observed_i, j] -= (H[j, t] - prev_H) * W_it
        A_filled = W @ H.T
        times.append(time.time()-start_time)
        RMSE_new = np.sqrt(np.mean((A_test[test] - A_filled[test]) ** 2))
        RMSE.append(RMSE_new)
        objective.append(np.linalg.norm(A[Omega]-A_filled[Omega])+lambda_*(np.linalg.norm(W,"fro"))+np.linalg.norm(H,"fro"))
        if np.linalg.norm(R - prev_R, ord='fro') < tol:
            break
    elapsed_time = time.time() - start_time
    return A_filled, np.log(objective), elapsed_time, times,RMSE

data_train = pd.read_csv('ratings-train.csv')
data_test = pd.read_csv('ratings-test.csv')
A_train = data_train.pivot(index="userId", columns="movieId", values="rating")
A_test = data_test.pivot(index="userId", columns="movieId", values="rating")
all_users = pd.Index.union(A_train.index, A_test.index)  
all_movies = pd.Index.union(A_train.columns, A_test.columns)  
A_train = A_train.reindex(index=all_users, columns=all_movies, fill_value=0)
A_test = A_test.reindex(index=all_users, columns=all_movies, fill_value=0)
A = A_train.fillna(0).to_numpy()
A_test = A_test.fillna(0).to_numpy()
Omega = A != 0  
test = A_test != 0  
A_test2=A_test.copy()
A2=A.copy()
k=15
lambda_=0.1
m, n = A.shape
np.random.seed(0)
C=np.random.rand(m, k)  
C2=C.copy()
D=np.random.rand(n, k) 
D2=D.copy()

A_filled_als,als_residuals,als_time,als_times,als_RMSE = ALS(C,D,A2,A_test2,Omega,test, k,lambda_,20)
A_filled_ccd,ccd_residuals, ccd_time,ccd_times,ccd_RMSE= CCD_MC(C2,D2,A,A_test,Omega,test, k,lambda_,20)

plt.figure(figsize=(10, 6))
plt.plot(ccd_residuals, label='CCD')
plt.plot(als_residuals, label='ALS')
plt.xlabel('k')
plt.ylabel('objective_value')
plt.title('objective over iteration: CCD vs ALS')
plt.legend()
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(ccd_times,ccd_residuals, label='CCD')
plt.plot(als_times,als_residuals, label='ALS')
plt.xlabel('time')
plt.ylabel('objective_value')
plt.title('objective over iteration: CCD vs ALS')
plt.legend()
plt.show()
print(f"Running time for CCD: {ccd_time:.2f} seconds")
print(f"Running time for ALS: {als_time:.2f} seconds")
print(f"RMSR for CCD: {ccd_RMSE[20]}")
print(f"RMSE for ALS: {als_RMSE[20]} ")


